# Exercise 1

Retrieve the <b>legal name</b>, the residence <b>country</b> and the <b>ultimate parent</b> (LEI and name) of all companies present in the provided datasets (*issuances.csv* and *holdings.csv*), and store them in a dataframe.

To structure the requests, please find "lei-records" endpoint details [GLEIF API](https://documenter.getpostman.com/view/7679680/SVYrrxuU?version=latest#5fc83577-f5ae-41fa-aa4e-2848554958b4).

---

In [ ]:
import pandas as pd
import requests
session = requests.Session()

In [ ]:
issuances_df = pd.read_csv('../data/issuances.csv', sep=';')
holdings_df = pd.read_csv('../data/holdings.csv')
url = "https://api.gleif.org/api/v1/lei-records/"

In [ ]:
children_companies = list(set(holdings_df["holder_lei"].values.tolist()))

In [ ]:
len(children_companies)

[ratelimit](https://pypi.org/project/ratelimit/)

In [ ]:
from ratelimit import limits, sleep_and_retry

# 60 calls per 60 seconds
CALLS = 60
RATE_LIMIT = 60

@sleep_and_retry
@limits(calls=CALLS, period=RATE_LIMIT)
def check_limit():
    ''' 
    Empty function just to check for calls to API 
    '''
    return

In [ ]:
result = []
for child in children_companies:
    check_limit()
    print(child)
    response_child = session.get(url + child , headers={"Accept": "application/vnd.api+json"})
    name = response_child.json()["data"]["attributes"]["entity"]["legalName"]["name"]
    country = response_child.json()["data"]["attributes"]["entity"]["legalAddress"]["country"]
    response_parent = session.get(url + child + "/" +  "ultimate-parent", headers={"Accept": "application/vnd.api+json"})
    ultimate_parent_lei = response_parent.json()["data"]["id"]
    ultimate_parent_name = response_parent.json()["data"]["attributes"]["entity"]["legalName"]["name"]
    result.append((child, name, country, ultimate_parent_lei, ultimate_parent_name))


In [ ]:
result_df = pd.DataFrame(result, columns=["child_lei", "child_name", "child_country", "parent_lei", "parent_name"])

In [ ]:
result_df

In [ ]:
result_df.to_csv('result_exercise_1.csv', index=False)